In [0]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from google.colab import files
from statsmodels.tsa.ar_model import AR
from random import random
from sklearn.metrics import mean_squared_error

# read the data file
# I ran the same block multiple times, just changing the name of the csv files to iterate through period 1~6
df = pd.read_csv('returns_6.csv',header=None)
data = df.to_numpy()
data = np.array(data).ravel()

train,test = train_test_split(data, train_size=0.9, test_size=0.1, shuffle=False)

# train model
model = AR(train)
# maxlag = 1 for AR (1) and maxlag = 20 for AR (20)
model_fit = model.fit(maxlag=1)
window = model_fit.k_ar
print(window) # check the number of autocorrelations being used
coef = model_fit.params 
# save the autocorrelation coefficients

# walk forward over time steps in test
history = train[len(train)-window:]
history = [history[i] for i in range(len(history))]
predictions = []

# AR makes predictions for the test period by using the autocorrelation coefficients acquired in training period
for t in range(len(test)):
	length = len(history)
	lag = [history[i] for i in range(length-window,length)]
	yhat = coef[0]
	for d in range(window):
		yhat += coef[d+1] * lag[window-d-1]
	obs = test[t]
	predictions=np.append(predictions,yhat)
	history.append(obs)
# error is measured as RMSE
error = mean_squared_error(test, predictions,squared=False)
print("RMSE: "+ str(error))


# same code as the second block in model performnace check
positive=0
negative=0
f_positive=0
f_negative=0
for i in range(len(predictions)):
    if predictions[i]>=0 and test[i]>=0:
        positive=positive+1
    elif predictions[i]>=0 and test[i]<0:
        f_positive= f_positive+1
    elif predictions[i]<0 and test[i]<0:
        negative=negative+1
    elif predictions[i]<0 and test[i]>=0:
        f_negative=f_negative+1
print (((positive+negative)/len(predictions))) #"Directional accuracy: "
print ((positive/(positive+f_positive))) #'+ accuracy: '
print ((negative/(negative+f_negative))) #'- accuracy: '
print (((positive+f_positive)/len(predictions))) #'% of + prediction: '

1
Test RMSE: 0.009
0.00913466852415072
0.49206349206349204
0.5411255411255411
0.41496598639455784
0.6111111111111112
